#  Matching Catalogs by Members ID

---
**License**

 Matching Catalogs by Members ID

 Tue Aug 24 10:30:00 2021\
 Copyright  2021\
 Cinthia Nunes de Lima <cinthia.n.lima@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 Ascaso_Extended_cDC2_RM_Test\
 Copyright (C) 2021 Cinthia Nunes de Lima <cinthia.n.lima@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

## Libraries & Configs

In [3]:
import math
import sys

import os
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.table import Table
import pandas as pd

sys.path.insert(0, "/global/homes/c/cinlima/gcr-catalogs")
sys.path.insert(0, "/global/homes/c/cinlima/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/MachingCatalogs")

import GCRCatalogs
#Put nersc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")

from numcosmo_py import Nc, Ncm

from mtdc2 import mtrmdc2

import time

## Catalogs

In [4]:
#Cosmo DC2
cosmodc2= GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')

#Cosmo DC2 + RedMaPPer
catalog_dc2_redmapper = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')

ImportError: cannot import name 'trapz' from 'scipy.integrate' (/global/homes/c/cinlima/.conda/envs/numcosmo/lib/python3.11/site-packages/scipy/integrate/__init__.py)

In [3]:
# Restrictions on richness, mass and redshift
min_richness = 5
min_halo_mass = 1e13 #Msun
redshift_max = 1.2

#cDC2 + RM data
quantities = catalog_dc2_redmapper.list_all_quantities()
cluster_quantities = [q for q in quantities if 'member' not in q]
member_quantities = [q for q in quantities if 'member' in q]
    
query = GCRCatalogs.GCRQuery('(richness > ' + str(min_richness) +')')
cluster_data = Table(catalog_dc2_redmapper.get_quantities(cluster_quantities, [query])) # cDC2 + RM cluster data
member_data = Table(catalog_dc2_redmapper.get_quantities(member_quantities)) # cDC2 + RM member data

#cDC2 data
quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central']
query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +') & (is_central < ' + str(redshift_max) +') ')
truth_data = Table(cosmodc2.get_quantities(quantities_wanted, [query]))

## Matching cDC2 and cDC2 + RM Catalogs

In [4]:
memberdf = member_data.to_pandas().rename(columns={'id_member': 'id'}) #This creates a dataframe and rename: id_member -> id (cDC2 + RM MEMBERS).
truthdf = truth_data.to_pandas().rename(columns={'galaxy_id': 'id'}) #This creates a dataframe and rename: galaxy_id -> id (cDC2 MEMBERS).

cluster_df=cluster_data.to_pandas()[['richness', 'cluster_id', 'redshift']] #This creates a dataframe for cDC2 clusters with columns:
                                                                            #                    richness', 'cluster_id', 'redshift.

In [5]:
memberdf

,magerr_r_lsst_member,magerr_i_lsst_member,p_member,magerr_g_lsst_member,id,refmag_err_member,magerr_z_lsst_member,cluster_id_member,mag_i_lsst_member,theta_r_member,...,dec_member,magerr_y_lsst_member,ra_member,refmag_member,mag_g_lsst_member,theta_i_member,mag_z_lsst_member,mag_r_lsst_member,redshift_true_member,mag_y_lsst_member
0,0.009540,0.013971,0.995835,0.009604,1312945563,0.009682,0.009682,10,17.899111,1.000000,...,-25.717996,0.010202,52.520436,17.505138,20.286411,1.000000,17.505138,18.798338,0.510724,17.322586
1,0.008483,0.009098,0.987648,0.012352,1312932492,0.009760,0.009760,10,18.054710,1.000000,...,-25.717833,0.009909,52.519490,17.676914,20.368668,1.000000,17.676914,18.879597,0.513088,17.482000
2,0.016074,0.015518,0.969744,0.018912,1312932655,0.014939,0.014939,10,20.486027,1.000000,...,-25.717501,0.017637,52.518707,20.064653,22.917450,1.000000,20.064653,21.444391,0.523674,19.879288
3,0.013284,0.017457,0.996897,0.027897,1312946401,0.015855,0.015855,10,21.054300,1.000000,...,-25.716904,0.021821,52.518979,20.635675,23.408300,1.000000,20.635675,21.914343,0.510740,20.473616
4,0.014500,0.016403,0.996004,0.016594,1312946359,0.013794,0.013794,10,20.303442,1.000000,...,-25.717097,0.019811,52.523239,19.945850,22.737461,1.000000,19.945850,21.228529,0.510501,19.697416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887513,0.064180,0.042129,0.229096,0.123047,12001566320,0.041282,0.041282,74968,23.514240,0.917139,...,-44.561452,0.096194,70.459703,22.905439,26.151848,0.999992,22.905439,24.625832,0.846997,22.761690
887514,0.025417,0.022025,0.514862,0.043481,12001731074,0.027570,0.027570,74968,22.219019,0.990446,...,-44.605494,0.036398,70.469214,21.535057,24.863094,1.000000,21.535057,23.265871,0.871953,21.283115
887515,0.021433,0.025786,0.734919,0.035801,12001996934,0.023381,0.023381,74968,22.173378,0.555642,...,-44.581587,0.034860,70.435240,21.391592,24.721514,1.000000,21.391592,23.223753,0.919259,21.121309
887516,0.050695,0.035155,0.387311,0.067558,12001996929,0.040786,0.040786,74968,23.527422,0.234176,...,-44.587221,0.047749,70.432336,22.717440,26.070425,1.000000,22.717440,24.613369,0.918583,22.449701


In [6]:
truthdf

,redshift,dec,halo_id,halo_mass,ra,is_central,id
0,0.048187,-40.400894,4500151475,1.689136e+13,49.636603,True,9437500128
1,0.049201,-40.413717,4500151475,1.689136e+13,49.465219,False,9437500129
2,0.048255,-40.405253,4500151475,1.689136e+13,49.629525,False,9437500130
3,0.049084,-40.402134,4500151475,1.689136e+13,49.637223,False,9437500131
4,0.047159,-40.406618,4500151475,1.689136e+13,49.472151,False,9437500132
...,...,...,...,...,...,...,...
5786055,1.199893,-38.474080,2708300146224,6.399267e+13,73.510064,False,9128383264
5786056,1.197388,-38.478944,2708300146224,6.399267e+13,73.508583,False,9128383265
5786057,1.198055,-38.468050,2708300146224,6.399267e+13,73.511652,False,9128383266
5786058,1.199955,-38.791467,625600146219,5.216248e+13,75.581856,False,9128679743


In [7]:
cluster_df

,richness,cluster_id,redshift
0,190.706650,10,0.513513
1,179.128128,18,0.571270
2,124.027153,24,1.040338
3,167.336029,30,0.635185
4,111.914703,71,0.442303
...,...,...,...
45076,5.373964,29100,1.135933
45077,7.555426,97018,1.134814
45078,5.007845,97912,0.211521
45079,5.358709,37112,1.147501


In [8]:
Mt_df = pd.merge(memberdf['id'], truthdf['id'], how='inner', on=['id']) #This selects commom members ID from cDC2 and cDC2 + RM data
Mt_df

,id
0,1312945563
1,1312932492
2,1312932655
3,1312946401
4,1312946359
...,...
542900,12001980397
542901,12001980408
542902,12001731074
542903,12001996934


In [9]:
mt_member = pd.merge(memberdf, Mt_df, how='inner', on=['id']) #This selects the matching members using Mt_df IDs
mt_member

,magerr_r_lsst_member,magerr_i_lsst_member,p_member,magerr_g_lsst_member,id,refmag_err_member,magerr_z_lsst_member,cluster_id_member,mag_i_lsst_member,theta_r_member,...,dec_member,magerr_y_lsst_member,ra_member,refmag_member,mag_g_lsst_member,theta_i_member,mag_z_lsst_member,mag_r_lsst_member,redshift_true_member,mag_y_lsst_member
0,0.009540,0.013971,0.995835,0.009604,1312945563,0.009682,0.009682,10,17.899111,1.000000,...,-25.717996,0.010202,52.520436,17.505138,20.286411,1.000000,17.505138,18.798338,0.510724,17.322586
1,0.008483,0.009098,0.987648,0.012352,1312932492,0.009760,0.009760,10,18.054710,1.000000,...,-25.717833,0.009909,52.519490,17.676914,20.368668,1.000000,17.676914,18.879597,0.513088,17.482000
2,0.016074,0.015518,0.969744,0.018912,1312932655,0.014939,0.014939,10,20.486027,1.000000,...,-25.717501,0.017637,52.518707,20.064653,22.917450,1.000000,20.064653,21.444391,0.523674,19.879288
3,0.013284,0.017457,0.996897,0.027897,1312946401,0.015855,0.015855,10,21.054300,1.000000,...,-25.716904,0.021821,52.518979,20.635675,23.408300,1.000000,20.635675,21.914343,0.510740,20.473616
4,0.014500,0.016403,0.996004,0.016594,1312946359,0.013794,0.013794,10,20.303442,1.000000,...,-25.717097,0.019811,52.523239,19.945850,22.737461,1.000000,19.945850,21.228529,0.510501,19.697416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565682,0.044475,0.031738,0.487121,0.070071,12001980397,0.036225,0.036225,74968,23.415142,1.000000,...,-44.579165,0.056782,70.468730,22.536907,26.244192,1.000000,22.536907,24.378717,0.935693,22.200325
565683,0.052621,0.042934,0.569421,0.074509,12001980408,0.044205,0.044205,74968,23.866930,1.000000,...,-44.574802,0.056838,70.461551,23.000544,26.395853,0.969526,23.000544,24.845156,0.936611,22.676394
565684,0.025417,0.022025,0.514862,0.043481,12001731074,0.027570,0.027570,74968,22.219019,0.990446,...,-44.605494,0.036398,70.469214,21.535057,24.863094,1.000000,21.535057,23.265871,0.871953,21.283115
565685,0.021433,0.025786,0.734919,0.035801,12001996934,0.023381,0.023381,74968,22.173378,0.555642,...,-44.581587,0.034860,70.435240,21.391592,24.721514,1.000000,21.391592,23.223753,0.919259,21.121309


In [10]:
idc = mt_member[['id', 'cluster_id_member']].rename(columns={'cluster_id_member': 'cluster_id'}) #This selects the members and clusters IDs
idc

,id,cluster_id
0,1312945563,10
1,1312932492,10
2,1312932655,10
3,1312946401,10
4,1312946359,10
...,...,...
565682,12001980397,74968
565683,12001980408,74968
565684,12001731074,74968
565685,12001996934,74968


In [11]:
mt_cluster = pd.merge(idc, cluster_df, how='inner', on=['cluster_id']) #This selects the clusters data by idc IDS 
mt_cluster

,id,cluster_id,richness,redshift
0,1312945563,10,190.706650,0.513513
1,1312932492,10,190.706650,0.513513
2,1312932655,10,190.706650,0.513513
3,1312946401,10,190.706650,0.513513
4,1312946359,10,190.706650,0.513513
...,...,...,...,...
565682,12001980397,74968,5.129885,0.918480
565683,12001980408,74968,5.129885,0.918480
565684,12001731074,74968,5.129885,0.918480
565685,12001996934,74968,5.129885,0.918480


In [12]:
mt_catalog = pd.merge(mt_cluster, truthdf, how='inner', on=['id']) #This creates the matching catalog with clustes and halos

In [13]:
clusters_id = mt_catalog[mt_catalog['is_central'] == True]['cluster_id'].unique() #This selects the clusters IDs

clusters_id

array([   10,    18,    24, ..., 80120, 21720, 54206], dtype=int32)

In [14]:
mt_catalog['freq'] = mt_catalog.groupby(['cluster_id', 'halo_id'])['halo_id'].transform('count') # This Adds a frequency column for each halo ID
                                                                                                 # in mt_catalog.
mt_catalog[0:15]

,id,cluster_id,richness,redshift_x,redshift_y,dec,halo_id,halo_mass,ra,is_central,freq
0,1312945563,10,190.70665,0.513513,0.510724,-25.717996,1162500021323,5.461543e+14,52.520436,True,96
1,1312932492,10,190.70665,0.513513,0.513088,-25.717833,205900021323,5.461543e+14,52.519490,True,86
2,1312932655,10,190.70665,0.513513,0.523674,-25.717501,205900021323,5.461543e+14,52.518707,False,86
3,1312946401,10,190.70665,0.513513,0.510740,-25.716904,1162500021323,5.461543e+14,52.518979,False,96
4,1312946359,10,190.70665,0.513513,0.510501,-25.717097,1162500021323,5.461543e+14,52.523239,False,96
5,1312932712,10,190.70665,0.513513,0.518737,-25.717318,205900021323,5.461543e+14,52.517520,False,86
6,1312945743,10,190.70665,0.513513,0.510393,-25.715388,1162500021323,5.461543e+14,52.522205,False,96
7,1312932517,10,190.70665,0.513513,0.509923,-25.717773,205900021323,5.461543e+14,52.524499,False,86
8,1312946270,10,190.70665,0.513513,0.512285,-25.718986,205900021323,5.461543e+14,52.516412,False,86
9,1312946366,10,190.70665,0.513513,0.515361,-25.719177,1162500021323,5.461543e+14,52.524799,False,96


In [15]:
iscentral = mt_catalog[mt_catalog['is_central'] == True] #This selects the central members

iscentral

,id,cluster_id,richness,redshift_x,redshift_y,dec,halo_id,halo_mass,ra,is_central,freq
0,1312945563,10,190.706650,0.513513,0.510724,-25.717996,1162500021323,5.461543e+14,52.520436,True,96
1,1312932492,10,190.706650,0.513513,0.513088,-25.717833,205900021323,5.461543e+14,52.519490,True,86
33,1313029113,10,190.706650,0.513513,0.568313,-25.727465,711400021315,8.179136e+13,52.519101,True,6
165,1312930910,10,190.706650,0.513513,0.508254,-25.716806,80500021323,1.158516e+13,52.472525,True,2
198,2000524808,18,179.128128,0.571270,0.574030,-27.204341,549500032315,1.009535e+15,56.920377,True,189
...,...,...,...,...,...,...,...,...,...,...,...
565610,11315892952,6928,17.501669,1.135227,1.168442,-43.144912,4041900181224,6.707982e+13,69.227644,True,18
565634,11315022342,80120,5.251925,1.075930,1.024822,-43.254448,1645300181241,2.216870e+13,66.977340,True,2
565646,11940608972,21720,10.819990,1.133446,1.131637,-45.049689,2358500191230,7.055515e+13,67.806866,True,13
565661,12000459039,54206,5.282434,0.501281,0.506876,-44.819294,958400192323,1.931144e+13,70.472970,True,6


In [16]:
# This loop creates the unique matching catalog using the halo ID frequency. It selects the halos with maximum frequency.
match_dataframe = pd.DataFrame()                                                                                                                                                                            #
for cl in clusters_id:                                                                                        
    gcut = iscentral.groupby('cluster_id').get_group(cl)
    match_dataframe = pd.concat([match_dataframe, gcut[gcut['freq'] == gcut['freq'].max()]], ignore_index=True)


## Time measurement

In [17]:
inicio = time.time()

mt_dataframe = mtrmdc2(member_data, cluster_data, truth_data)

fim = time.time()
tempo_total = fim - inicio

print(f"Tempo de execução: {tempo_total:.3}s")

Tempo de execução: 1.03e+03s
